In [283]:
import pyvo as vo
from pyvo import dal, registry
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import astropy as ap
import polars as pl
from typing import List, Tuple, Dict, Union, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from dataclasses import dataclass


In [284]:
# Define the cone search parameters
ra = 10.68479
dec = 41.26906
radius = 0.1

# Convert to SkyCoord
coord = SkyCoord(ra, dec, unit='deg')
coord

<SkyCoord (ICRS): (ra, dec) in deg
    (10.68479, 41.26906)>

In [381]:

# Get a list of all the services which support cone searches and covers the area we are interested in
resources = registry.search(servicetype='conesearch', spatial=coord)

print(len(resources))

1945


In [277]:
test_res = resources[1].search(coord, radius*u.deg)

In [299]:
type(test_res)

pyvo.dal.scs.SCSResults

In [376]:

def cone_search(resource: vo.registry.regtap.RegistryResource, coord, radius) -> Table:
    """ Perform a cone search on a service.

    Args:
        service (vo.registry.regtap.RegistryResource): The service to search
        coord (astropy.coordinates.SkyCoord): The center of the cone search
        radius (float): The radius of the cone search in degrees
    
    Returns:
        Table: The results of the cone search
    """
    return Table.from_cone_search(resource, coord, radius)

In [353]:
test_res = cone_search(services[800], coord, radius)

In [355]:
test_res

Table(name=J/other/RAA/19.6,
title=LAMOST spectroscopic binaries & variable stars description=About 786.4 thousand stars were observed by LAMOST twice or more during the first stage of its spectroscopic survey. The radial velocity differences for about 256 thousand targets are larger than 10km/s and they are possible spectroscopic binary or variable candidates (SBVCs). It is shown that most SBVCs are slightly metal poorer than the Sun. There are two peaks in the temperature distribution of SBVCs around 5760K and 4870K, while there are three peaks in the distribution of the gravitational acceleration at 2.461, 4.171 and 4.621cm/s^2^. The locations of SBVCs on the [Fe/H]-T, [Fe/H]-logg, logg-T and H-R diagrams are investigated. It is found that the detected SBVCs could be classified into four groups. The first group has higher logg~4.621 and lower T~4870K which are mainly cool red dwarf binaries. The second group of SBVCs has logg around 4.171cm/s^2^ that includes binaries and pulsating 

In [354]:
test_res.data

_r,recno,Name,Obs.start,Obs.end,N,Teff,e_Teff,logg,e_logg,[Fe/H],e_[Fe/H],RVlow,e_RVlow,RVhigh,e_RVhigh,DRV,e_DRV,_RA,_DE
f64,i64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.087854,13162,"""C004218.65+411...","""2011-10-28""","""2012-12-04""","""2""",5507.069824,121.080002,3.754,0.172,-0.817,0.113,-267.790009,11.55,-233.130005,17.48,34.66,20.95,10.57771,41.23389
0.043902,128928,"""C004245.11+411...","""2013-10-16""","""2013-11-12""","""2""",5372.089844,40.299999,4.406,0.056,0.086,0.038,-95.860001,7.84,-81.059998,4.03,14.8,8.82,10.68796,41.22522


In [356]:
import os
# clear the test folder
if os.path.exists('test_table'):
    for file in os.listdir('test_table'):
        os.remove(os.path.join('test_table', file))
    os.rmdir('test_table')
    
os.mkdir('test_table')
test_res.save('test_table')

In [361]:
# try to load the table
test_res2 = Table.load('test_table')

In [362]:
test_res2

Table(name=J/other/RAA/19.6,
title=LAMOST spectroscopic binaries & variable stars description=About 786.4 thousand stars were observed by LAMOST twice or more during the first stage of its spectroscopic survey. The radial velocity differences for about 256 thousand targets are larger than 10km/s and they are possible spectroscopic binary or variable candidates (SBVCs). It is shown that most SBVCs are slightly metal poorer than the Sun. There are two peaks in the temperature distribution of SBVCs around 5760K and 4870K, while there are three peaks in the distribution of the gravitational acceleration at 2.461, 4.171 and 4.621cm/s^2^. The locations of SBVCs on the [Fe/H]-T, [Fe/H]-logg, logg-T and H-R diagrams are investigated. It is found that the detected SBVCs could be classified into four groups. The first group has higher logg~4.621 and lower T~4870K which are mainly cool red dwarf binaries. The second group of SBVCs has logg around 4.171cm/s^2^ that includes binaries and pulsating 

In [363]:
test_res2.metadata

TableMetadata(access_url='http://vizier.cds.unistra.fr/viz-bin/conesearch/J/other/RAA/19.64/table1?', name='J/other/RAA/19.6', title='LAMOST spectroscopic binaries & variable stars', description='About 786.4 thousand stars were observed by LAMOST twice or more during the first stage of its spectroscopic survey. The radial velocity differences for about 256 thousand targets are larger than 10km/s and they are possible spectroscopic binary or variable candidates (SBVCs). It is shown that most SBVCs are slightly metal poorer than the Sun. There are two peaks in the temperature distribution of SBVCs around 5760K and 4870K, while there are three peaks in the distribution of the gravitational acceleration at 2.461, 4.171 and 4.621cm/s^2^. The locations of SBVCs on the [Fe/H]-T, [Fe/H]-logg, logg-T and H-R diagrams are investigated. It is found that the detected SBVCs could be classified into four groups. The first group has higher logg~4.621 and lower T~4870K which are mainly cool red dwarf 

In [364]:
resources = services

In [377]:
def cone_search_list(resource_list, coord, radius, wait_between: int = 0) -> List[Table]:
    """ Access all resources at a given url with an optional delay between each request.
    
    Args:
        resources (List[vo.registry.regtap.RegistryResource]): The resources to search
        coord (astropy.coordinates.SkyCoord): The center of the cone search
        radius (float): The radius of the cone search in degrees
        wait_between (int): The number of seconds to wait between each request

    Returns:
        List[Table]
    """
    results = []
    for resource in resource_list:
        results.append(cone_search(resource, coord, radius))
        time.sleep(wait_between)
    return results

In [380]:
len(resources)

1

In [382]:
source_folder = 'source'

resources_by_access_url = dict()
for access_url in set(resource.access_url for resource in resources):
    resources_by_access_url[access_url] = tuple(resource for resource in resources if resource.access_url == access_url)    


with ThreadPoolExecutor(max_workers=len(resources_by_access_url)) as executor:
    futures = []
    for access_url, resource_list in resources_by_access_url.items():
        futures.append(executor.submit(cone_search_list, resource_list, coord, radius, wait_between=1))
    
    for future in as_completed(futures, timeout=500):
        try:
            for table in future.result():
                save_path = f'{source_folder}//{table.metadata.access_url.replace(r"/", "__s__")}//{table.name}'
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                
                table.save(save_path)
        except Exception as e:
            try:
                os.mkdir('errors')
            except FileExistsError:
                pass
            try:
                with open(f'errors/{table.name.replace(r"/", "__s__")}.txt', 'a') as f:
                    f.write(str(e))
            except Exception as e2:
                print(f'Writing error to file failed: {e2}')   
                print(f'Error: {e}')
            


    

In [391]:
# load all the tables from the source folder
tables = []
for root, dirs, files in os.walk(source_folder):
    for file in files:
        if file == 'metadata.json':
            try:
                tables.append(Table.load(root))
            except Exception as e:
                print(f'Error loading table: {root}')
                print(e)

len(tables)

1072

In [393]:
len([table for table in tables if table.metadata is not None])

1072

In [389]:
[t.fields for t in tables]

[[FieldMetadata(name='_r', datatype='double', description='Distance from center ( 10.684790+41.269060)[FK5/J2000]', unit=None, ucd='pos.angDistance', arraysize=None, xtype=None, ref=None),
  FieldMetadata(name='recno', datatype='int', description='Record number assigned by the VizieR team. Should Not be used for identification.', unit=None, ucd='meta.record', arraysize=None, xtype=None, ref=None),
  FieldMetadata(name='GLADE+', datatype='char', description='GLADE+ catalog number [datatype=int]', unit=None, ucd='ID_MAIN', arraysize='8', xtype=None, ref=None),
  FieldMetadata(name='PGC', datatype='int', description='? Principal Galaxies Catalogue number [NULL integer written as an empty string]', unit=None, ucd='meta.id', arraysize=None, xtype=None, ref=None),
  FieldMetadata(name='GWGC', datatype='char', description='Name in the GWGC catalog', unit=None, ucd='meta.id', arraysize='28', xtype=None, ref=None),
  FieldMetadata(name='HyperLEDA', datatype='char', description='Name in the Hype

In [188]:
columns_df.groupby('table_name')\
    .agg('column_name')\
    .filter(pl.col('column_name').arr.eval(pl.element().str.contains('RAJ2000')).arr.contains(True).is_not())\
    .sort('table_name', descending=True)

table_name,column_name
str,list[str]
"""I/304""","[""_r"", ""CMC14"", … ""2MASS""]"
"""I/271""","[""_r"", ""GSC2.2"", … ""Np""]"
"""I/259""","[""_r"", ""recno"", … ""CCDM""]"
"""I/243""","[""_r"", ""USNO-A1.0"", … ""Epoch""]"
"""I/239""","[""_r"", ""recno"", … ""_DE.icrs""]"


In [67]:
columns_df.head()

table_name,column_name
str,str
"""I/306A""","""_RAJ2000"""
"""I/306A""","""_DEJ2000"""
"""I/306A""","""_r"""
"""I/306A""","""recno"""
"""I/306A""","""Seq"""


In [201]:
columns_df.groupby('table_name')\
          .agg(pl.col('column_name'))\
          .filter(pl.col('column_name').arr.contains('RA(ICRS)') & 
                  (pl.col('column_name').arr.contains('_RAJ2000') ^ pl.col('column_name').arr.contains('RAJ2000'))
                  .is_not())\
          .explode('column_name')\
          .groupby('column_name')\
          .agg(pl.count('table_name'))\
          .sort('table_name', descending=True)


column_name,table_name
str,u32
"""RA(ICRS)""",3
"""_r""",3
"""DE(ICRS)""",3
"""e_DEdeg""",2
"""e_RAdeg""",2
"""Epoch""",2
"""Rmag""",2
"""e_BTmag""",1
"""pmDE""",1


In [206]:
# Now we know the RA and Dec columns are called RAJ2000 and DEJ2000 or _RAJ2000 and _DEJ2000
# We can use this to standardize the coordinate column names
# We can also use this to standardize the coordinate units

def standardize_coordinates(df: pl.DataFrame) -> pl.DataFrame:
    """
    Standardize the coordinate columns in a DataFrame to be named RAJ2000 and DEJ2000 and in degrees
    """
    # Check if both the '_' versions and the non '_' versions of the coordinate columns are present
    if all(col in df.columns for col in ["RAJ2000", "DEJ2000"]) and all(col in df.columns for col in ["_RAJ2000", "_DEJ2000"]):
        # If both are present, drop the '_' versions
        df = df.drop(["_RAJ2000", "_DEJ2000"])
        
    elif all(col in df.columns for col in ["_RAJ2000", "_DEJ2000"]):
        df = df.rename({"_RAJ2000": "RAJ2000", "_DEJ2000": "DEJ2000"})

    elif all(col in df.columns for col in ["RAICRS", "DEICRS"]):
        df = df.rename({"RAICRS": "RAJ2000", "DEICRS": "DEJ2000"})

    elif all(col in df.columns for col in ["RA(ICRS)", "DE(ICRS)"]):
        df = df.rename({"RA(ICRS)": "RAJ2000", "DE(ICRS)": "DEJ2000"})
    

    try:
        # Rename the coordinate columns
        df = df.rename({"RAJ2000": "RA", "DEJ2000": "DE"})
    except Exception as e:
        print('Problem with renaming columns')
        print(type(e))
        print(e)
        print(df.columns)    


    # Convert the units
    for col in ["RA", "DE"]:
        if col in df.columns:
            
            # some of the columns are strings, so we need to convert them to floats
            if df[col].dtype == pl.Utf8 and col == 'DE':
                # assume the DE column format is "Degrees ArcMinutes ArcSeconds"
                de_parts = df.select(pl.col(col).str.split(' ').alias('de_parts'))
                de_parts = de_parts.with_columns([
                    pl.col('de_parts').arr[0].str.starts_with('+').alias('positive'),
                    pl.col('de_parts').arr[0].cast(pl.Int16).alias('de_deg'),
                    pl.col('de_parts').arr[1].cast(pl.Int16).alias('de_arcmin'),
                    pl.col('de_parts').arr[2].cast(pl.Float64).alias('de_arcsec')
                    ])
                
                # convert to degrees. arcmin and arcsec are converted to degrees by dividing by 60 and 3600
                de_parts = de_parts.with_columns(
                    pl.when(pl.col('positive'))
                      .then(pl.col('de_deg') + pl.col('de_arcmin') / 60 + pl.col('de_arcsec') / 3600)
                      .otherwise(pl.col('de_deg') - pl.col('de_arcmin') / 60 - pl.col('de_arcsec') / 3600)
                      .alias(f'DE')
                    )
                de_parts = de_parts.drop(['de_parts', 'positive', 'de_deg', 'de_arcmin', 'de_arcsec'])
                df = df.drop(col)
                df = df.select('*', de_parts.to_series())

            if df[col].dtype == pl.Utf8 and col == 'RA':
                # assume the RA column format is "Hours Minutes Seconds"
                ra_parts = df.select(pl.col(col).str.split(' ').alias('ra_parts'))
                ra_parts = ra_parts.with_columns([
                    pl.col('ra_parts').arr[0].cast(pl.Int16).alias('ra_hrs'),
                    pl.col('ra_parts').arr[1].cast(pl.Int16).alias('ra_min'),
                    pl.col('ra_parts').arr[2].cast(pl.Float64).alias('ra_sec')
                    ])
                
                # min and sec are converted to hours by dividing by 60 and 3600
                # then multiply by 15 to convert to degrees
                ra_parts = ra_parts.with_columns(
                    ((pl.col('ra_hrs') + pl.col('ra_min') / 60 + pl.col('ra_sec') / 3600) * 15)
                    .alias(f'RA')
                    )
                ra_parts = ra_parts.drop(['ra_parts', 'ra_hrs', 'ra_min', 'ra_sec'])
                df = df.drop(col)
                df = df.select('*', ra_parts.to_series())

                
            try:
                df = df.with_columns(df[col].cast(pl.Float64).alias(f'{col}_deg'))
            except Exception as e:
                print(type(e))
                print(e)
                print(df[col])
                print(df[col].dtype)
                
    
    return df

In [208]:
standardized_results = {table: standardize_coordinates(df) for table, df in results.items()}

In [231]:
len(standardized_results[list(standardized_results.keys())[0]])

3

In [213]:
# Now we try to find columns that identify unique objects
# We can use this to standardize the object identifier column names

def standardize_object_id(df: pl.DataFrame) -> pl.DataFrame:
    
    total_rows = df.select(pl.count(df.columns[0])).item()
    # Find the number of unique values in each column
    unique = df.select(pl.all().n_unique())
    
    return unique

standardize_object_id(standardized_results[list(standardized_results.keys())[0]])

_r,recno,Seq,m_Seq,ci,ref1,ref2,ref3,ref4,ref5,ref6,ref7,ref8,ref9,u_RA2000,u_pmRA,pmRA,n_pmRA,u_DE2000,pmDE,Epoch,f_Bmag,Bmag,n_Bmag,f_Vmag,Vmag,n_Vmag,f_Umag,Umag,n_Umag,f_Rmag,Rmag,n_Rmag,f_Imag,Imag,n_Imag,f_Jmag,Jmag,f_Hmag,Hmag,f_Kmag,Kmag,f_[Fe/H],[Fe/H],f_RV,RV,Sp,n_Sp,Sp2,n_Sp2,f_Lum,Lum,var,mul,n_mul,n_PA,PA,n_Sep,Sep,FONAK1.0,u_AC2000,AC2000,u_UCAC2,UCAC2,u_GSC,GSC,Ids,RA,RA_deg,DE,DE_deg
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
3,3,3,1,1,1,2,1,1,1,1,1,1,2,2,2,3,2,2,3,3,1,2,2,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,2,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3


In [209]:
# Now we can collect the results into a single DataFrame
# We can also add a column to indicate which table the results came from

all_results = pl.concat([df.with_columns(pl.lit(table).alias('source_table')) for table, df in standardized_results.items()])

ShapeError: unable to append to a dataframe of width 72 with a dataframe of width 19

In [130]:
len(results)

52

In [40]:
# The following gets a little bit more complicated.
# Some pre-reading: https://en.wikipedia.org/wiki/Astronomical_coordinate_systems
# https://en.wikipedia.org/wiki/Equatorial_coordinate_system
# https://en.wikipedia.org/wiki/Galactic_coordinate_system
# https://en.wikipedia.org/wiki/Ecliptic_coordinate_system


I/306A: ['_RAJ2000', '_DEJ2000', '_r', 'recno', 'Seq', 'm_Seq', 'ci', 'ref1', 'ref2', 'ref3', 'ref4', 'ref5', 'ref6', 'ref7', 'ref8', 'ref9', 'u_RA2000', 'RAJ2000', 'u_pmRA', 'pmRA', 'n_pmRA', 'u_DE2000', 'DEJ2000', 'pmDE', 'Epoch', 'f_Bmag', 'Bmag', 'n_Bmag', 'f_Vmag', 'Vmag', 'n_Vmag', 'f_Umag', 'Umag', 'n_Umag', 'f_Rmag', 'Rmag', 'n_Rmag', 'f_Imag', 'Imag', 'n_Imag', 'f_Jmag', 'Jmag', 'f_Hmag', 'Hmag', 'f_Kmag', 'Kmag', 'f_[Fe/H]', '[Fe/H]', 'f_RV', 'RV', 'Sp', 'n_Sp', 'Sp2', 'n_Sp2', 'f_Lum', 'Lum', 'var', 'mul', 'n_mul', 'n_PA', 'PA', 'n_Sep', 'Sep', 'FONAK1.0', 'u_AC2000', 'AC2000', 'u_UCAC2', 'UCAC2', 'u_GSC', 'GSC', 'Ids']
I/275: ['_RAJ2000', '_DEJ2000', '_r', 'recno', 'RAJ2000', 'DEJ2000', 'Bmag', 'mEp', 'Num', 'e_RAJ2000', 'e_DEJ2000', 'ACnum', 'HIP', 'm_HIP', 'TYC2', 'Vmag', 'magflg', 'verflg']
I/207: ['_RAJ2000', '_DEJ2000', '_r', 'recno', 'TICID1', 'TICID2', 'RAJ2000', 'DEJ2000', 'ePos', 'e_Bmag', 'Bmag', 'e_Vmag', 'Vmag', 'Flag1', 'Flag2', 'Flag3', 'Flag4', 'Flag5', 'Flag

In [ ]:



def estimate_coord_system(df: pl.DataFrame) -> str:
    """ From a polars DataFrame, estimate the coordinate system used.

    Args:
        df: A polars DataFrame from a cone search

    Returns:
        A tuple containing the coordinate system/s used in the cone search.
            Possible values are 'equatorial', 'galactic', 'ecliptic'
    """
    coord_systems = set()
    for col in df.columns:
        if col.casefold() in col_names_to_coord_systems:
            coord_systems.add(col_names_to_coord_systems[col.casefold()])
    return tuple(coord_systems)


# Constants for the equatorial to galactice conversion
# https://en.wikipedia.org/wiki/Equatorial_coordinate_system#Converting_equatorial_coordinates_to_galactic_coordinates
RA0 = 192.859508
DEC0 = 27.128336
L0 = 32.932
RA0_RAD = RA0 * u.deg.to(u.rad)
DEC0_RAD = DEC0 * u.deg.to(u.rad)
L0_RAD = L0 * u.deg.to(u.rad)



def convert_equatorial_to_galactic(ra_expr: pl.Expr, da_expr: pl.Expr) -> Tuple[pl.Expr, pl.Expr]:
    """ Given two polars expressions, return the expressions for the galactic coordinates.

    This function uses exclusively polars expressions and does not use astropy or numpy as it is
    intended to be used in a polars DataFrame abd polars expressions are the fastest way to do.

    The math can be found here: https://en.wikipedia.org/wiki/Equatorial_coordinate_system#Converting_equatorial_coordinates_to_galactic_coordinates
        
    Args:
        ra_expr: A polars expression for the right ascension
        da_expr: A polars expression for the declination
    
    Returns:
        A tuple containing the expressions for the galactic longitude and latitude (l, b)
    """
    # Convert the angles to radians
    ra_rad = ra_expr * u.deg.to(u.rad)
    dec_rad = da_expr * u.deg.to(u.rad)
    
    # Calculate the galactic longitude
    l = pl.lit(L0_RAD) + pl.atan2(
        pl.sin(ra_rad - pl.lit(RA0_RAD)) * pl.cos(dec_rad),
        pl.cos(ra_rad - pl.lit(RA0_RAD)) * pl.sin(dec_rad) * pl.cos(pl.lit(DEC0_RAD)) -
        pl.sin(dec_rad) * pl.sin(pl.lit(DEC0_RAD))
    )
    
    # Calculate the galactic latitude
    b = pl.asin(
        pl.sin(dec_rad) * pl.cos(pl.lit(DEC0_RAD)) +
        pl.cos(dec_rad) * pl.sin(pl.lit(DEC0_RAD)) * pl.sin(ra_rad - pl.lit(RA0_RAD))
    )
    
    # Convert the angles back to degrees
    return l * u.rad.to(u.deg), b * u.rad.to(u.deg)

# Constants for the ecliptic to galactice conversion
# https://ned.ipac.caltech.edu/forms/calculator.html




def convert_ecl


def add_galactic_coords(df: pl.DataFrame) -> pl.DataFrame:
    """ Check the coordinate system of the cone search and add a column with the galactic coordinates
    if the coordinate system is not galactic.
    
    Args:
        df: A polars DataFrame from a cone search

    Returns:
        A polars DataFrame with the galactic coordinates added
    """
    coord_systems = estimate_coord_system(df)
    if not coord_systems:
        raise ValueError("No coordinate system found")
    if "galactic" not in coord_systems:
        # Check for any coordinate system can be converted to galactic coordinates
        galactic_coords = None
        if "equatorial" in coord_systems:
            # Find the equatorial columns
            ra_col_name = [col for col in df.columns if col.casefold().startswith("ra")][0]
            dec_col_name = [col for col in df.columns if col.casefold().startswith("dec") or col.casefold().startswith("dej")][0]
            # Convert the equatorial coordinates to galactic coordinates
            l, b = convert_equatorial_to_galactic(pl.col(ra_col_name), pl.col(dec_col_name))
            # Add the coordinates to the DataFrame
            df = df.with_column(pl.col("l").alias("glon"))
            df = df.with_column(pl.col("b").alias("glat"))
        elif "ecliptic" in coord_systems:
            # Find the ecliptic columns
            elon_col_name = [col for col in df.columns if col.casefold().startswith("elon")][0]
            elat_col_name = [col for col in df.columns if col.casefold().startswith("elat")][0]
            # Convert the ecliptic coordinates to galactic coordinates
            l, b = convert_equatorial_to_galactic(pl.col(elon_col_name), pl.col(elat_col_name))
            # Add the coordinates to the DataFrame
            df = df.with_column(pl.col("l").alias("glon"))
            df = df.with_column(pl.col("b").alias("glat"))
    return df


In [37]:
# Convert the coordinates to galactic coordinates
converted_results = {}
for service, df in results.items():
    try:
        converted_results[service] = add_galactic_coords(df)
    except ValueError as e:
        print(e)
        # Check the columns in the DataFrame
        print(df.columns)

No coordinate system found
['_RAJ2000', '_DEJ2000', '_r', 'recno', 'Seq', 'm_Seq', 'ci', 'ref1', 'ref2', 'ref3', 'ref4', 'ref5', 'ref6', 'ref7', 'ref8', 'ref9', 'u_RA2000', 'RAJ2000', 'u_pmRA', 'pmRA', 'n_pmRA', 'u_DE2000', 'DEJ2000', 'pmDE', 'Epoch', 'f_Bmag', 'Bmag', 'n_Bmag', 'f_Vmag', 'Vmag', 'n_Vmag', 'f_Umag', 'Umag', 'n_Umag', 'f_Rmag', 'Rmag', 'n_Rmag', 'f_Imag', 'Imag', 'n_Imag', 'f_Jmag', 'Jmag', 'f_Hmag', 'Hmag', 'f_Kmag', 'Kmag', 'f_[Fe/H]', '[Fe/H]', 'f_RV', 'RV', 'Sp', 'n_Sp', 'Sp2', 'n_Sp2', 'f_Lum', 'Lum', 'var', 'mul', 'n_mul', 'n_PA', 'PA', 'n_Sep', 'Sep', 'FONAK1.0', 'u_AC2000', 'AC2000', 'u_UCAC2', 'UCAC2', 'u_GSC', 'GSC', 'Ids']
No coordinate system found
['_RAJ2000', '_DEJ2000', '_r', 'recno', 'RAJ2000', 'DEJ2000', 'Bmag', 'mEp', 'Num', 'e_RAJ2000', 'e_DEJ2000', 'ACnum', 'HIP', 'm_HIP', 'TYC2', 'Vmag', 'magflg', 'verflg']
No coordinate system found
['_RAJ2000', '_DEJ2000', '_r', 'recno', 'TICID1', 'TICID2', 'RAJ2000', 'DEJ2000', 'ePos', 'e_Bmag', 'Bmag', 'e_Vmag',

In [38]:
converted_results['I/322A']

_r,UCAC4,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,ePos,EpRA,EpDE,f.mag,a.mag,e_a.mag,of,db,Na,Nu,Nc,pmRA,e_pmRA,pmDE,e_pmDE,MPOS1,UCAC2,Tycho-2,2Mkey,Jmag,e_Jmag,q_Jmag,Hmag,e_Hmag,q_Hmag,Kmag,e_Kmag,q_Kmag,Bmag,e_Bmag,f_Bmag,Vmag,e_Vmag,f_Vmag,gmag,e_gmag,f_gmag,rmag,e_rmag,f_rmag,imag,e_imag,f_imag,g,c,H,A,b,h,Z,B,L,N,S,LEDA,2MX
f64,str,f64,i64,f64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,str,str,i64,f64,f64,str,f64,f64,str,f64,f64,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,f64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0.094608,"""656-002659""",10.620926,139,41.187551,135,202,1994.37,1993.95,15.919,16.42,NaN,1,0,1,1,2,7.7,7.2,-3.6,6.9,154523915,"""""","""""",422825567,15.275,0.06,"""05""",14.794,0.09,"""05""",14.503,0.11,"""05""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",0,0,0,0,0,0,0,0,0,1,0,0,0
0.099356,"""656-002684""",10.692983,67,41.169896,83,106,2000.93,1999.8,15.929,15.722,0.13,1,0,3,3,2,6.8,7.1,-9.6,6.9,154505804,"""""","""""",1287661032,14.515,0.03,"""05""",14.139,0.05,"""05""",14.104,0.06,"""05""",16.645,"""14""","""-""",15.972,"""01""","""-""",16.278999,"""01""","""-""",15.952,"""01""","""-""",NaN,"""""","""""",0,0,0,0,0,0,0,0,0,1,0,0,0
0.08913,"""656-002686""",10.699059,30,41.180578,22,39,2001.92,2002.24,14.534,14.611,0.08,1,0,5,4,2,-0.5,4.9,-12.0,4.6,154516745,"""""","""""",1287660957,13.648,0.03,"""05""",13.367,0.03,"""05""",13.372,0.04,"""05""",15.197,"""08""",""".""",14.632,"""03""",""".""",14.857,"""00""",""".""",14.539,"""02""",""".""",14.518,"""05""",""".""",0,0,0,0,0,0,0,0,0,1,0,0,0
0.094413,"""656-002693""",10.714214,20,41.177278,22,33,2002.41,2002.32,14.13,14.086,0.05,1,0,5,5,2,-3.1,5.4,-1.0,5.3,154513362,"""""","""""",1287660976,12.956,0.02,"""05""",12.62,0.02,"""05""",12.52,0.02,"""05""",14.894,"""04""",""".""",14.196,"""04""",""".""",14.478,"""03""",""".""",14.016,"""04""",""".""",13.846,"""05""",""".""",0,0,0,0,0,0,0,0,0,1,0,0,0
0.095752,"""656-002712""",10.763501,53,41.193798,51,73,1999.45,1999.72,15.586,15.492,0.13,1,0,3,3,2,22.9,4.2,-7.4,4.2,154530445,"""""","""""",1287660821,14.419,0.04,"""05""",14.156,0.04,"""25""",14.122,0.07,"""05""",16.07,"""11""",""".""",15.366,"""02""",""".""",15.618,"""07""",""".""",15.269,"""11""",""".""",15.265,"""07""",""".""",0,0,0,0,0,0,0,0,0,1,0,0,0
0.087178,"""657-002621""",10.572181,91,41.248229,144,177,1999.1,1992.75,16.318001,15.946,0.14,1,0,3,3,2,-4.4,6.9,8.9,7.0,154586501,"""""","""""",422824945,15.204,0.06,"""05""",14.803,0.09,"""05""",14.478,0.1,"""05""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",0,0,0,0,0,0,0,0,0,7,0,0,0
0.086984,"""657-002622""",10.5738,164,41.29375,160,354,1970.34,1969.47,15.905,NaN,NaN,1,0,1,1,2,0.6,6.4,-3.7,6.3,154633934,"""""","""""",422824421,15.63,0.08,"""05""",15.227,0.17,"""07""",15.008,0.17,"""07""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",0,0,0,0,0,0,0,0,0,1,0,0,0
0.086957,"""657-002623""",10.575487,255,41.240611,209,378,2002.65,2002.65,16.528999,NaN,NaN,5,0,1,1,1,NaN,NaN,NaN,NaN,154578695,"""""","""""",422825042,15.496,0.99,"""02""",14.884,0.11,"""26""",14.722,0.99,"""02""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",NaN,"""""","""""",0,0,0,0,0,0,0,0,0,0,0,0,0
0.087829,"""657-002624""",10.577732,39,41.23391,45,62,2002.07,2001.82,14.911,15.086,0.2,1,0,3,3,2,-11.6,7.1,-5.9,6.8,154571778,"""""","""""",422825125,13.644,0.03,"""05""",13.158,0.03,"""05""",13.038,0.04,"""05""",15.647,"""12""",""".""",14.873,"""08""",""".""",15.256,"""08""",""".""",14.687,"""09""",""".""",14.491,"""25""",""".""",0,0,0,0,0,0,0,0,0,1,0,0,0


In [39]:
len(converted_results)

2